In [1]:
from sklearn import tree
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneGroupOut
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import warnings; warnings.simplefilter('ignore')

### reading data and printing the data ###
data = pd.read_csv('RAW_CGM_Signal.csv',names=['ID','TruncatedID','Fat','Carb','Protein','c1','c2','c3','c4','c5','c6','c7','c8','c9','c10','c11','c12','c13','c14','c15','c16','c17','c18','c19','c20','c21','c22','c23','c24','c25','c26','c27','c28','c29'
])
# data = data.dropna()
print(data.shape)
print(list(data.columns))


### done with reading data and printing the data ###

(165, 34)
['ID', 'TruncatedID', 'Fat', 'Carb', 'Protein', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13', 'c14', 'c15', 'c16', 'c17', 'c18', 'c19', 'c20', 'c21', 'c22', 'c23', 'c24', 'c25', 'c26', 'c27', 'c28', 'c29']


In [2]:
CHO_data = data.drop(['TruncatedID','Carb','Fat'], axis=1) #taking only the Carb target column
print(CHO_data.Protein.unique())# Finding the unique values in that column
print(CHO_data.ID.unique()) #finiding unique IDs

[ 9 11 18]
["    '2133-001'" "    '2133-002'" "    '2133-004'" "    '2133-006'"
 "    '2133-008'" "    '2133-009'" "    '2133-010'" "    '2133-011'"
 "    '2133-012'" "    '2133-013'" "    '2133-015'" "    '2133-017'"
 "    '2133-018'" "    '2133-019'" "    '2133-020'" "    '2133-021'"
 "    '2133-022'" "    '2133-024'" "    '2133-025'" "    '2133-026'"
 "    '2133-028'" "    '2133-030'" "    '2133-032'" "    '2133-033'"
 "    '2133-035'" "    '2133-036'" "    '2133-037'" "    '2133-039'"
 "    '2133-040'" "    '2133-041'"]


In [3]:
## Low-High

CHO_LH_data = CHO_data[CHO_data['Protein'].isin([9,18])]
CHO_LH_data.loc[CHO_LH_data.Protein == 9, 'CHO_bin'] = 0
CHO_LH_data.loc[CHO_LH_data.Protein== 18, 'CHO_bin'] = 1
CHO_LH_data = CHO_LH_data.drop(['Protein'], axis=1)

X = CHO_LH_data.loc[:, CHO_LH_data.columns != 'CHO_bin']
X =X.drop(['ID'], axis=1)
Y = CHO_LH_data.loc[:, CHO_LH_data.columns == 'CHO_bin']
X = X.values
Y = Y.values
# # X = np.array(df2[features])
# # Y = np.array(df2['label'])
groups = np.array(CHO_LH_data['ID'])

# #clf = tree.DecisionTreeClassifier()
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(12), random_state=1)
logo = LeaveOneGroupOut()

print (logo.get_n_splits(X, Y, groups))

y_pred = [-1 for x in range(len(Y))]

for train_index, test_index in logo.split(X, Y, groups=groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    clf.fit(X_train,y_train)
    y_pred_subject = clf.predict(X_test)
    j = 0
    for i in test_index:
        y_pred[i] = y_pred_subject[j]
        j+=1

print (y_pred)

print (confusion_matrix(Y, y_pred))

tn, fp, fn, tp = confusion_matrix(Y, y_pred).ravel()

print ("True Negatives ", tn)
print ("False Positives ", fp)
print ("False Negatives ", fn)
print ("True Positives ", tp)

yes_acc = tp/float(tp+fn)
no_acc = tn/float(tn+fp)
print ("class accuracies :", yes_acc, no_acc)
un_acc = 0.5*(tp/float(tp+fn)) + 0.5*(tn/float(tn+fp))

print ("Unweighted accuracy :", un_acc)
print ("\n\n built in accuracy",accuracy_score(Y, y_pred))

"""
scores = cross_val_score(clf, X, Y, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
"""


30
[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]
[[21 33]
 [14 42]]
True Negatives  21
False Positives  33
False Negatives  14
True Positives  42
class accuracies : 0.75 0.3888888888888889
Unweighted accuracy : 0.5694444444444444


 built in accuracy 0.5727272727272728


'\nscores = cross_val_score(clf, X, Y, cv=10)\nprint("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))\n'

In [4]:
## Medium-High

CHO_LH_data = CHO_data[CHO_data['Protein'].isin([11,18])]
CHO_LH_data.loc[CHO_LH_data.Protein == 11, 'CHO_bin'] = 0
CHO_LH_data.loc[CHO_LH_data.Protein== 18, 'CHO_bin'] = 1
CHO_LH_data = CHO_LH_data.drop(['Protein'], axis=1)

X = CHO_LH_data.loc[:, CHO_LH_data.columns != 'CHO_bin']
X =X.drop(['ID'], axis=1)
Y = CHO_LH_data.loc[:, CHO_LH_data.columns == 'CHO_bin']
X = X.values
Y = Y.values
# # X = np.array(df2[features])
# # Y = np.array(df2['label'])
groups = np.array(CHO_LH_data['ID'])

# #clf = tree.DecisionTreeClassifier()
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(12), random_state=1)
logo = LeaveOneGroupOut()

print (logo.get_n_splits(X, Y, groups))

y_pred = [-1 for x in range(len(Y))]

for train_index, test_index in logo.split(X, Y, groups=groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    clf.fit(X_train,y_train)
    y_pred_subject = clf.predict(X_test)
    j = 0
    for i in test_index:
        y_pred[i] = y_pred_subject[j]
        j+=1

print (y_pred)

print (confusion_matrix(Y, y_pred))

tn, fp, fn, tp = confusion_matrix(Y, y_pred).ravel()

print ("True Negatives ", tn)
print ("False Positives ", fp)
print ("False Negatives ", fn)
print ("True Positives ", tp)

yes_acc = tp/float(tp+fn)
no_acc = tn/float(tn+fp)
print ("class accuracies :", yes_acc, no_acc)
un_acc = 0.5*(tp/float(tp+fn)) + 0.5*(tn/float(tn+fp))

print ("Unweighted accuracy :", un_acc)
print ("\n\n built in accuracy",accuracy_score(Y, y_pred))

"""
scores = cross_val_score(clf, X, Y, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
"""


30
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[[ 7 48]
 [ 6 50]]
True Negatives  7
False Positives  48
False Negatives  6
True Positives  50
class accuracies : 0.8928571428571429 0.12727272727272726
Unweighted accuracy : 0.5100649350649351


 built in accuracy 0.5135135135135135


'\nscores = cross_val_score(clf, X, Y, cv=10)\nprint("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))\n'

In [5]:
## Low-Medium

CHO_LH_data = CHO_data[CHO_data['Protein'].isin([9,11])]
CHO_LH_data.loc[CHO_LH_data.Protein == 9, 'CHO_bin'] = 0
CHO_LH_data.loc[CHO_LH_data.Protein== 11, 'CHO_bin'] = 1
CHO_LH_data = CHO_LH_data.drop(['Protein'], axis=1)

X = CHO_LH_data.loc[:, CHO_LH_data.columns != 'CHO_bin']
X =X.drop(['ID'], axis=1)
Y = CHO_LH_data.loc[:, CHO_LH_data.columns == 'CHO_bin']
X = X.values
Y = Y.values
# # X = np.array(df2[features])
# # Y = np.array(df2['label'])
groups = np.array(CHO_LH_data['ID'])

# #clf = tree.DecisionTreeClassifier()
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(12), random_state=1)
logo = LeaveOneGroupOut()

print (logo.get_n_splits(X, Y, groups))

y_pred = [-1 for x in range(len(Y))]

for train_index, test_index in logo.split(X, Y, groups=groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    clf.fit(X_train,y_train)
    y_pred_subject = clf.predict(X_test)
    j = 0
    for i in test_index:
        y_pred[i] = y_pred_subject[j]
        j+=1

print (y_pred)

print (confusion_matrix(Y, y_pred))

tn, fp, fn, tp = confusion_matrix(Y, y_pred).ravel()

print ("True Negatives ", tn)
print ("False Positives ", fp)
print ("False Negatives ", fn)
print ("True Positives ", tp)

yes_acc = tp/float(tp+fn)
no_acc = tn/float(tn+fp)
print ("class accuracies :", yes_acc, no_acc)
un_acc = 0.5*(tp/float(tp+fn)) + 0.5*(tn/float(tn+fp))

print ("Unweighted accuracy :", un_acc)
print ("\n\n built in accuracy",accuracy_score(Y, y_pred))

"""
scores = cross_val_score(clf, X, Y, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
"""


30
[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
[[30 24]
 [34 21]]
True Negatives  30
False Positives  24
False Negatives  34
True Positives  21
class accuracies : 0.38181818181818183 0.5555555555555556
Unweighted accuracy : 0.4686868686868687


 built in accuracy 0.46788990825688076


'\nscores = cross_val_score(clf, X, Y, cv=10)\nprint("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))\n'